In [2]:
# !pip install -U datasets

In [3]:
# !sudo -H pip install -q transformers --upgrade

In [5]:
# !pip uninstall autoawq -y

In [ ]:
# !pip install autoawq

In [7]:
# !sudo -H pip install https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.7/autoawq-0.1.7+cu118-cp38-cp38-linux_x86_64.whl

In [9]:
# !pip install transformers==4.35.2

In [1]:
import pandas
import numpy
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [2]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer,AutoModelForCausalLM

In [3]:
import torch
torch.__version__

'2.0.1+cu117'

In [4]:
#model_path = '/data/mistral/query-to-mql/exp-10/nov-20/merged-model'

In [5]:
quant_path = '/llmmodels/quantized_model/awq_mistral'

In [6]:
model_path='mistralai/Mistral-7B-v0.1'

In [7]:
quant_config = {"zero_point":True,
               "q_group_size": 128,
               "w_bit": 4,
               "version": "GEMM"
               }

In [8]:
#load model
# model = AutoAWQForCausalLM.from_pretrained(model_path, **{"low_cpu_mem_usage":True})
model = AutoAWQForCausalLM.from_pretrained(model_path, device_map='auto')

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

In [10]:
import torch
torch.cuda.empty_cache()

In [11]:
import gc
gc.collect()


31

In [ ]:
# torch.cuda.memory_summary(device=None, abbreviated=False)

In [12]:
#Quantize
# dont run it again
model.quantize(tokenizer, quant_config=quant_config)

/tmp/pip_packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
AWQ: 100%|██████████| 32/32 [41:02<00:00, 76.96s/it]


In [ ]:
# model_quantized = AutoModelForCausalLM.from_pretrained(
#     model,
#     torch_dtype=torch.float16,
#     device_map="auto",
#     quantization_config=quant_config
# )

In [13]:
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

('/llmmodels/quantized_model/awq_mistral/tokenizer_config.json',
 '/llmmodels/quantized_model/awq_mistral/special_tokens_map.json',
 '/llmmodels/quantized_model/awq_mistral/tokenizer.model',
 '/llmmodels/quantized_model/awq_mistral/added_tokens.json',
 '/llmmodels/quantized_model/awq_mistral/tokenizer.json')

### Inference on quantized model

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
quant_path = '/llmmodels/quantized_model/awq_mistral'
model_path='mistralai/Mistral-7B-v0.1'

In [9]:
model_id = quant_path
quant_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

def predict_from_quant(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = quant_model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

In [10]:
import time

In [11]:
# Using quant model
start = time.time()
output1 = predict_from_quant("what is art")
print("time taken is :", time.time()-start)

time taken is : 42.05787110328674


In [15]:
%%timeit
output1 = predict_from_quant("what is art")

40.9 s ± 44.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
print(output1)

<s> what is art?

I’ve been thinking about this a lot lately.

I’ve been thinking about it because I’ve been reading a lot of books about art.

I’ve been thinking about it because I’ve been reading a lot of books about art.

I’ve been thinking about it because I’ve been reading a lot of books about art.

I’ve been thinking about it because I’ve been reading a lot of books about art.

I’ve been thinking about it because I’ve been reading a lot of books about art.

I’ve been thinking about it because I’ve been reading a lot of books about art.

I’ve been thinking about it because I’ve been reading a lot of books about art.

I’ve been thinking about it because I’ve been reading a lot of books about art.

I’ve been thinking about it because I’ve been reading a lot of books about art.

I’ve been thinking about it because I’ve been reading a lot of books about art.

I’ve been thinking about it because I’ve been reading a lot of books about art.

I’ve been thinking about it because I’ve been 

### Inference on un-quantized model

In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
# model_path = '/data/quantization-trials/merged-model'/

In [17]:
model_id = model_path
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

def predict_from_normal(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB (GPU 0; 15.60 GiB total capacity; 14.61 GiB already allocated; 164.94 MiB free; 14.75 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# Using unquant model
import time
start = time.time()
output2 = predict_from_normal("what is art")
print("time taken is :", time.time()-start)

In [ ]:
print(output2)

In [ ]:
model

In [ ]:
quant_model